# Backdoor AI - Ollama Llama3 Groq Tool Use Model Downloader

This notebook downloads the llama3-groq-tool-use model using Ollama and zips it for easy download. This notebook is designed to work with Google Colab.

## How it works

1. Check if we're running in Google Colab
2. Optimize the Colab environment for large models
3. Install Ollama and necessary dependencies
4. Download the llama3-groq-tool-use model using Ollama
5. Zip the model files for easy download
6. Provide a download link for the zipped model


## Check if running in Google Colab

First, let's make sure we're running in Google Colab.

In [ ]:
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    print("⚠️ This notebook is designed to run in Google Colab. Some features may not work as expected.")
else:
    print("✅ Running in Google Colab!")
    
    # Import Google Colab specific modules
    from google.colab import drive, files

## Memory Optimization for Large Models

First, let's clear up disk space and optimize memory to ensure we have enough resources for large models.

In [ ]:
# Memory optimization functions
import os
import shutil
import subprocess
import gc
import time
from IPython.display import display, HTML, clear_output

# Install required packages first
!pip install -q psutil tqdm
import psutil

def clear_disk_space():
    """Clean up disk space by removing unnecessary files."""
    print("🧹 Cleaning up disk space...")
    
    # Clean apt cache
    subprocess.run("apt-get clean", shell=True)
    
    # Remove unnecessary packages
    subprocess.run("apt-get -y autoremove", shell=True)
    
    # Clean pip cache
    subprocess.run("rm -rf ~/.cache/pip", shell=True)
    
    # Remove temporary files
    temp_dirs = ['/tmp', '/var/tmp']
    for temp_dir in temp_dirs:
        if os.path.exists(temp_dir):
            try:
                for item in os.listdir(temp_dir):
                    item_path = os.path.join(temp_dir, item)
                    # Skip our ollama directories
                    if item.startswith('ollama') or item.startswith('backdoor'):
                        continue
                    
                    try:
                        if os.path.isdir(item_path):
                            shutil.rmtree(item_path)
                        else:
                            os.remove(item_path)
                    except Exception as e:
                        pass  # Skip files that can't be removed
            except Exception as e:
                print(f"Warning: Could not clean {temp_dir}: {e}")
    
    # Remove unused Docker images/containers if Docker is installed
    try:
        subprocess.run("docker system prune -af", shell=True, stderr=subprocess.DEVNULL)
    except:
        pass
    
    print("✅ Disk cleanup complete!")
    
    # Show disk space
    show_disk_usage()

def show_disk_usage():
    """Show current disk usage."""
    try:
        df_output = subprocess.check_output("df -h /", shell=True, text=True)
        print("\n📊 Disk Space Available:")
        for line in df_output.split('\n'):
            print(line)
    except:
        print("Could not retrieve disk usage information")

def show_memory_usage():
    """Show current memory usage."""
    try:
        memory = psutil.virtual_memory()
        total_gb = memory.total / (1024 ** 3)
        available_gb = memory.available / (1024 ** 3)
        used_gb = memory.used / (1024 ** 3)
        percent = memory.percent
        
        print(f"\n📊 Memory Usage:")
        print(f"Total Memory: {total_gb:.2f} GB")
        print(f"Available: {available_gb:.2f} GB")
        print(f"Used: {used_gb:.2f} GB ({percent}%)")
    except:
        print("Could not retrieve memory usage information")

def clear_memory():
    """Clear Python memory."""
    gc.collect()
    torch_available = False
    
    try:
        import torch
        torch_available = True
    except ImportError:
        pass
    
    if torch_available:
        try:
            import torch
            torch.cuda.empty_cache()
            print("✅ PyTorch CUDA cache cleared")
        except:
            pass
    
    print("✅ Python memory cleared")

# Run optimization process
print("🚀 Optimizing environment for large language models...")
clear_disk_space()
clear_memory()

# Set environment variables for improved performance
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

# Show current resource usage
show_memory_usage()
show_disk_usage()

print("\n✅ Optimization complete! Ready to continue.")

## Install Ollama

Now let's install Ollama and other required dependencies.

In [ ]:
# Install Ollama
# Install GPU detection tools first
!apt-get update && apt-get install -y lspci lshw pciutils

print("💾 Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# Install other dependencies
!pip install -q requests httpx

# Set up directories
!mkdir -p /tmp/ollama/models

print("✅ Ollama installed successfully!")

## Start Ollama Server

Now we'll start the Ollama server in the background.

In [ ]:
import subprocess
import time
import requests
import json
from IPython.display import clear_output

# Start Ollama server in background
ollama_process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for Ollama to start
print("Starting Ollama server...")
time.sleep(5)

# Check if Ollama is running
try:
    response = requests.get("http://localhost:11434/api/version", timeout=30)
    if response.status_code == 200:
        print(f"✅ Ollama is running! Version: {response.json().get('version')}")
    else:
        print(f"❌ Ollama returned unexpected status: {response.status_code}")
except Exception as e:
    print(f"❌ Failed to connect to Ollama: {e}")
    print("Trying to start again...")
    # Kill the previous process if it exists
    if ollama_process:
        ollama_process.terminate()
        time.sleep(2)
    # Try starting again
    !ollama serve &
    time.sleep(5)
    try:
        response = requests.get("http://localhost:11434/api/version", timeout=30)
        if response.status_code == 200:
            print(f"✅ Second attempt succeeded! Ollama is running. Version: {response.json().get('version')}")
    except:
        print("❌ Failed to start Ollama after multiple attempts.")

## Download the Llama3 Groq Tool Use Model

Now let's download the llama3-groq-tool-use model using Ollama.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess
import time

# Set the model ID for llama3-groq-tool-use
model_id = "llama3-groq-tool-use:8b"
output_area = widgets.Output()

# Function to download model
def download_model():
    with output_area:
        print(f"🚀 Downloading llama3-groq-tool-use model: {model_id}")
        print("This may take a while depending on your internet connection...")
        print("You'll see progress below. Please don't interrupt the process.")
        
        # Run ollama pull command
        process = subprocess.Popen(
            ["ollama", "pull", model_id],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        
        # Show live output
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                # Add sleep to prevent CPU overload
                time.sleep(0.1)
                break
            if output:
                print(output.strip())
        
        return_code = process.poll()
        if return_code == 0:
            print(f"✅ Model {model_id} downloaded successfully!")
            # List available models
            print("\n📋 Available models:")
            !ollama list
        else:
            print(f"❌ Failed to download model {model_id}. Return code: {return_code}")

# Download the model
download_model()
display(output_area)

## Zip the Ollama Model Files

Now let's zip the model files for easy download.

In [ ]:
import os
import shutil
import glob

# Define the model name and paths
model_name = "llama3-groq-tool-use-8b"
ollama_model_dir = "/root/.ollama/models"
zip_dir = "/content"
zip_file = f"{zip_dir}/{model_name}.zip"

# Create a temporary directory to store the model files
temp_dir = f"{zip_dir}/{model_name}"
os.makedirs(temp_dir, exist_ok=True)

print(f"Copying model files to {temp_dir}...")

# Find all files related to the model
model_files = []
for root, dirs, files in os.walk(ollama_model_dir):
    for file in files:
        if "llama3-groq-tool-use" in file.lower():
            model_files.append(os.path.join(root, file))

# Copy model files to the temporary directory
for file_path in model_files:
    file_name = os.path.basename(file_path)
    dest_path = os.path.join(temp_dir, file_name)
    shutil.copy2(file_path, dest_path)
    print(f"  - Copied {file_name}")

# Also copy the model manifest file if it exists
manifest_file = os.path.join(ollama_model_dir, "manifests", "registry.ollama.ai", "library", "llama3-groq-tool-use")
if os.path.exists(manifest_file):
    manifest_dir = os.path.join(temp_dir, "manifest")
    os.makedirs(manifest_dir, exist_ok=True)
    shutil.copy2(manifest_file, os.path.join(manifest_dir, "manifest"))
    print(f"  - Copied model manifest")

# Zip the model directory
print(f"\nZipping model files to {zip_file}...")
shutil.make_archive(f"{zip_dir}/{model_name}", 'zip', zip_dir, model_name)

# Get the size of the zip file
if os.path.exists(zip_file):
    zip_size = os.path.getsize(zip_file) / (1024 * 1024 * 1024)  # Convert to GB
    print(f"✅ Model zipped successfully to {zip_file}")
    print(f"Zip file size: {zip_size:.2f} GB")
else:
    print(f"❌ Failed to create zip file at {zip_file}")

## Download the Zipped Model

Now you can download the zipped model to your local machine.

In [ ]:
if IN_COLAB:
    print("Preparing download...")
    from google.colab import files
    files.download(zip_file)
    print("✅ Download initiated. If it doesn't start automatically, check your browser's download manager.")
else:
    print(f"⚠️ Not running in Colab. The zipped model is available at {zip_file}")

## Optional: Clean Up

If you want to free up space after downloading, you can run this cell to remove the temporary files.

In [ ]:
# Uncomment and run this cell if you want to remove the temporary files
# import shutil
# shutil.rmtree(temp_dir)
# print(f"✅ Removed temporary model files at {temp_dir}")